<div id="top">包括以下操作：</div>
<li>划分数据集。</li>
<li><a href="#2">数据集匹配分类。</a></li>
<li><a href="#2-2">循环遍历文件夹，进行数据集匹配分类。</a></li>
<li><a href="#3">统计各类别数量生成csv文件。</a></li>
<li><a href="#4">相同名称文件夹的数据合并到一起。</a></li>

### 划分数据集

按比例划分

In [ ]:
# split_dataset_ratio
# coding=utf-8
import os, random, shutil
import glob

def moveFile(fileDir, ratio, tarDir):
    pathDir = os.listdir(fileDir)  # 取图片的原始路径
    filenumber = len(pathDir)
    print(fileDir, filenumber)
    picknumber = int(filenumber * ratio)  # 按照rate比例从文件夹中取一定数量图片
    print('picknumber', picknumber)
    sample = random.sample(pathDir, picknumber)  # 随机选取picknumber数量的样本图片
    for name in sample:
        shutil.move(os.path.join(fileDir, name), os.path.join(tarDir, name))
    return

def listD():
    root = os.getcwd()
    ori_path = os.path.join(root, '分类数据').replace('\\', '/')  # 最开始train的文件夹路径
    split_Dir = os.path.join(root, 'train').replace('\\', '/')  # 移动到新的文件夹路径
    ratio = 0.8  # 抽取比例
    ori_path = glob.escape(ori_path)
    for filePath in os.listdir(ori_path):
        fileDir = os.path.join(ori_path, filePath)  # 原图片文件夹路径
        tarDir = os.path.join(split_Dir, filePath)  # val下子文件夹名字
        # print(tarDir)
        if not os.path.exists(tarDir):  # 如果val下没有子文件夹，就创建
            os.makedirs(tarDir)
        moveFile(fileDir, ratio, tarDir)  # 从每个子类别开始逐个划分

if __name__ == '__main__':
    listD()

按具体数量划分

In [ ]:

# split_dataset_number
# coding=utf-8
import os, random, shutil
import glob

def moveFile(fileDir, tarDir):
    pathDir = os.listdir(fileDir)  # 取图片的原始路径
    picknumber = 555
    print(fileDir)
    sample = random.sample(pathDir, picknumber)  # 随机选取picknumber数量的样本图片
    for name in sample:
        shutil.move(os.path.join(fileDir, name), os.path.join(tarDir, name))
    return

def listD():
    root = os.getcwd()
    ori_path = os.path.join(root, 'eggDastaset').replace('\\', '/')  # 最开始train的文件夹路径
    split_Dir = os.path.join(root, 'train').replace('\\', '/')  # 移动到新的文件夹路径
    ori_path = glob.escape(ori_path)
    for filePath in os.listdir(ori_path):
        fileDir = os.path.join(ori_path, filePath)  # 原图片文件夹路径
        tarDir = os.path.join(split_Dir, filePath)  # val下子文件夹名字
        if not os.path.exists(tarDir):  # 如果val下没有子文件夹，就创建
            os.makedirs(tarDir)
        moveFile(fileDir, tarDir)  # 从每个子类别开始逐个划分

if __name__ == '__main__':
    listD()

### <a id="2">数据集匹配分类</a>

<a href='#top'>▲ Top</a>

In [ ]:
# combine_match_data.py
# 遍历不同文件，提取匹配的文件，如 xml 和其对应的 jpg
import os
import shutil

outputfile = open('output.txt', 'w')

def iter_files(rootDir, oriRoot):
    newFolderPath = os.path.join(oriRoot, 'data')

    if not os.path.isdir(newFolderPath): # 如果目标不存在，则创建
        os.makedirs(newFolderPath)

    # newFolderPath_trainIMG = os.path.join(oriRoot, 'train/images').replace('\\','/')
    # newFolderPath_trainLABEL = os.path.join(oriRoot, 'train/labels').replace('\\','/')
    # PathList = [newFolderPath_trainIMG, newFolderPath_trainLABEL]

    # for path in PathList:
    #     if not os.path.isdir(path): # 如果目标不存在，则创建
    #         os.makedirs(path)

    for root,dirs,files in os.walk(rootDir): # root 就是文件存放的文件夹, file就是对应的文件
        fileSplitList = []
        for file in files:
            if any(file.lower().endswith(ext) for ext in ['.xml']):
                fileName, suffix = os.path.splitext(file)
                fileSplitList.append(fileName)
                oldFilePath = os.path.join(root,file).replace('\\','/') # 旧文件路径
                print('oldFilePath', oldFilePath)

                # 以下代码二选一
                shutil.copy(oldFilePath, newFolderPath)
                outputfile.write(oldFilePath)
                outputfile.write('\r\n')

                # 不会修改原文件里的文件名，只会修改复制后的文件名称
                # symbolCat = [FoldName, file]
                # newFileName = ('_').join(symbolCat)
                # print('newFileName', newFileName)
                # shutil.copyfile(oldFilePath, os.path.join(newFolderPath_trainLABEL, newFileName)) # oldFilePath 移动到 一个新的文件路径

        for file in files:
            if any(file.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']): 
                fileName, suffix = os.path.splitext(file)
                if fileName in fileSplitList: 
                    oldFilePath = os.path.join(root,file).replace('\\','/') # 旧文件路径
                    
                    # 以下代码二选一
                    shutil.copy(oldFilePath, newFolderPath)
                    print('oldFilePath', oldFilePath)

                    # symbolCat = [FoldName, file]
                    # newFileName = ('_').join(symbolCat)
                    # print('newFileName', newFileName)
                    # shutil.copyfile(oldFilePath, os.path.join(newFolderPath_trainIMG, newFileName))

oriRoot = os.getcwd()
rootDir = os.path.join(oriRoot, '王泰龙的切片').replace('\\','/')
iter_files(rootDir, oriRoot)
outputfile.close()

In [ ]:
# split_match_data.py
import shutil
import glob
import random
import os

def train_val(image_list, oriRoot):
    imageNumber = len(image_list)
    PathList = []

    train_image_dir = "images/train/"
    val_image_dir = "images/val/"

    train_label_dir = "labels/train/"
    val_label_dir = "labels/val/"

    train_txt_dir = 'train/txt/'
    val_txt_dir = 'val/txt/'

    PathList.append([val_image_dir, val_label_dir])

    set_val_number = int(0.8 * imageNumber)
    print('set_val_number', set_val_number)
    val_list = random.sample(image_list, set_val_number)
    for val in val_list:
        # val_image_name = val.split("/")[-1]
        val_image_name = val.split("\\")[-1]
        val_image_path = val_image_dir + val_image_name
        train_image_path = train_image_dir + val_image_name

        val_label_path = val_label_dir + val_image_name[:-3] + 'xml'
        train_label_path = train_label_dir + val_image_name[:-3] + 'xml'

        for path in PathList[0]:
            path = os.path.join(oriRoot, path).replace('\\', '/')
            if not os.path.isdir(path): # 如果目标不存在，则创建
                os.makedirs(path)

        shutil.move(train_image_path, val_image_path) # 原本所有文件都放在 训练集文件夹 train 里，取一部分放到 测试集 文件夹 val 里
        shutil.move(train_label_path, val_label_path)

        # txt 文件本来就是不匹配的
        # val_txt_path = val_txt_dir + val_image_name[:-3] + 'txt'
        # train_txt_path = train_txt_dir + val_image_name[:-3] + 'txt'
        # shutil.move(train_txt_path, val_txt_path)


if __name__ == '__main__':
    # image_list = glob.glob(r'/train/images/*.jpg')
    oriRoot = os.getcwd()
    path = os.path.join(oriRoot, 'images/train').replace('\\', '/') # path 是 原始放置图片的文件。无关 xml 文件存放位置
    glob.escape(path)
    image_list = glob.glob(os.path.join(glob.escape(path), '*.jpg'))
    print('image_list', image_list)
    train_val(image_list, oriRoot)

### <a id="2-2">循环遍历文件夹，进行数据集匹配分类</a>

<a href='#top'>▲ Top</a>

In [ ]:
import os
import random
import glob
import json
import shutil
from xml.etree import ElementTree as ET


def json_split_train_and_val(train_percent, val_percent, annotation_path, train_and_val_txt_path):
    json_annotation_path = annotation_path
    total_json = os.listdir(json_annotation_path)
    random.shuffle(total_json)

    num_json = len(total_json)
    num_train = int(train_percent * num_json)
    num_val = int(val_percent * num_json)
    train_list = total_json[:num_train]
    val_list = total_json[num_train:]

    for i in train_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "train")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.move(src_path, dst_path)

    for i in val_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "val")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.move(src_path, dst_path)


def xml_split_train_and_val(train_percent, val_percent, annotation_path, train_and_val_txt_path):
    xml_annotation_path = annotation_path
    total_xml = os.listdir(xml_annotation_path)
    random.shuffle(total_xml)

    num_xml = len(total_xml)
    num_train = int(train_percent * num_xml)
    num_val = int(val_percent * num_xml)
    train_list = total_xml[:num_train]
    val_list = total_xml[num_train:]

    for i in train_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "train")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.copy(src_path, dst_path)

    for i in val_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "val")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.copy(src_path, dst_path)


def move_jpg_xml(total_jpg_dir, annotation_path, JPEGImages_path,images_path,labels_path, not_concat=True,delete_file=False):
    if not_concat:
        # 检查jpg文件夹是否存在且非空
        if os.path.exists(JPEGImages_path) and any(os.scandir(JPEGImages_path)):
            # 删除文件夹及其内容
            shutil.rmtree(JPEGImages_path)
            os.makedirs(JPEGImages_path)

        # 检查xml文件夹是否存在且非空
        if os.path.exists(annotation_path) and any(os.scandir(annotation_path)):
            # 删除文件夹及其内容
            shutil.rmtree(annotation_path)
            os.makedirs(annotation_path)

        # 检查images文件夹是否存在且非空
        if os.path.exists(images_path) and any(os.scandir(images_path)):
            # 删除文件夹及其内容
            shutil.rmtree(images_path)
            os.makedirs(images_path)

        # 检查labels文件夹是否存在且非空
        if os.path.exists(labels_path) and any(os.scandir(labels_path)):
            # 删除文件夹及其内容
            shutil.rmtree(labels_path)
            os.makedirs(labels_path)

    jpg_list = []
    xml_List = []
    for root,dirs,files in os.walk(total_jpg_dir):
        for img_i in files:
            if any(img_i.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']): # 对于所有为图片的文件，不管是哪个层级的

                xml_i = img_i.replace("jpg", "xml")
                img_i_path = os.path.join(root, img_i).replace('\\', '/')
                xml_i_path = os.path.join(root, xml_i).replace('\\', '/')
                print('img_i_path == ', img_i_path)
                print('xml_i_path == ', xml_i_path)
                if not os.path.exists(xml_i_path):
                    os.remove(img_i_path) # 里面是路径
                    # continue # 跳出本次循环，不会继续执行之后的代码
                else:
                    xml_List.append(xml_i_path)

    for xml_i_path in xml_List:
        xml_data = ET.parse(xml_i_path)
        img_i_path = xml_i_path.replace("xml", "jpg")
        root = xml_data.getroot()

        object_state = root.findall("object")
        if len(object_state) != 0:
            # 移动jpg
            jpg_name = os.path.basename(img_i_path) # 返回path最后的文件名
            xml_name = os.path.basename(xml_i_path)

            src_img_path = img_i_path
            dst_img_path = os.path.join(JPEGImages_path, jpg_name).replace('\\', '/')
            shutil.copy(src_img_path, dst_img_path)

            # 移动xml
            src_xml_path = xml_i_path
            dst_xml_path = os.path.join(annotation_path, xml_name).replace('\\', '/')
            shutil.copy(src_xml_path, dst_xml_path)
        else:
            # 如果没有标注框，就对图片和xml进行删除
            os.remove(img_i_path)
            os.remove(xml_i_path)



if __name__ == "__main__":
    train_percent = 0.8
    val_percent = 0.2
    annotation_path = r"Annotations"
    JPEGImages_path = r"JPEGImages"
    images_path = r"images"
    labels_path = r"labels"
    # total_jpg_dir = r"testData/**/*.jpg" # testData\\D1-C3\\id=73_35_Scort=20.2552.jpg
    # total_jpg_dir = r"testData/**/**/*.jpg" # testData\\test2\\D1-C2\\id=73_35_Scort=20.2552.jpg
    total_jpg_dir = r"testData"
    # total_jpg_dir = r"/gpfs/users/aiAnalyze/jilei/chicken_blood/立华/未验证/20240701以后不需要胡一泓验证直接训练，因此这些数据不再放进已验证/20240718立华/20240614-F511-512已看已改/**/*.jpg"
    # total_jpg_dir = r"E:\DataSets\chicken_blood\立华\未验证\20240701以后不需要胡一泓验证直接训练，因此这些数据不再放进已验证\20240724_temp/*.jpg"

    move_jpg_xml(total_jpg_dir, annotation_path, JPEGImages_path,images_path,labels_path, not_concat=True,delete_file=False) # 第一次为 True，后续为 False，这样去合并文件
    # 功能：训练和验证数据划分，可等待move_jpg_xml函数汇总完数据后运行 # 全部数据汇总完毕后再运行
    # xml_split_train_and_val(train_percent, val_percent, annotation_path, JPEGImages_path)


### <a id="3">统计各类别数量生成csv文件</a> 

<a href='#top'>▲ Top</a>

In [ ]:
# coding=utf-8
import os, random, shutil
import glob
import pandas as pd


def listD():
    root = os.getcwd().replace('\\', '/')
    print('root', root)
    # ori_path = os.path.join(root, 'eggDataset/train/') # 最开始train的文件夹路径
    ori_path = root

    list = []
    dict = {"AnBanDan": "暗斑蛋", "BanDianDan": "斑点蛋", "BanZhuangDan": "板状蛋", "EAA": "EAA", "ShaPiDan": "沙皮蛋", "TaiYangDan": "太阳蛋", "YiXingDan": "异形蛋", 
            "ZangDan": "脏蛋", "ZhengChangDan": "正常蛋", "ZhouWenDan": "皱纹蛋"}
    column = ["类别", "数量"]
    for fileName in os.listdir(ori_path):
        if os.path.isdir(fileName):
            fileDir = os.path.join(ori_path, fileName)  # 原图片文件夹路径

            if fileName in dict:

                pathDir = os.listdir(fileDir)  # 取图片的原始路径
                filenumber = len(pathDir)
                fileDir = fileDir.replace('\\', '/').split('/')
                print(fileName, filenumber)


                item = [dict[fileName], filenumber]
                list.append(item)
                # 根据对应的关系，对列表名称进行修改

                # 将统计到的数据输出到 csv 文件
                # dict_to_df = {"类别": fileNameList, "数量": fileNumberList}
    
    print(list)
    df = pd.DataFrame(columns=column, data=list)
    df.to_csv("number_statistics.csv",index = False)

if __name__ == '__main__':
    listD()

### <a id="4">相同名称文件夹的数据合并到一起</a> 

<a href='#top'>▲ Top</a>


shutil.copy(oldFilePath, newFolderPath) 是旧文件路径，到新文件夹路径 ！！！ 否则会报错 PermissionError: [Errno 13] Permission denied

新目录和旧目录没有放在同一个目录下！！! 不然遍历的时候也会遍历新目录。

In [ ]:
# Same_Name_Folder_Combine
# coding=utf-8
import os, random, shutil
import glob

outputfilename = open('outputname.txt', 'w')

def CombineFile():
    rootDir = os.getcwd()
    for root,dirs,files in os.walk(rootDir):
        for dir in dirs:
            oldFolderPath = os.path.join(root, dir).replace('\\','/')

            # 新文件路径不要放在和旧文件同一个目录下
            newFolderPath = os.path.join('../dataset', dir).replace('\\','/')
            # print(newFolderPath)
            if not os.path.isdir(newFolderPath): # 如果目标不存在，则创建
                os.makedirs(newFolderPath)

            for file in os.listdir(oldFolderPath):
                if any(file.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']): # 对于所有为图片的文件，不管是哪个层级的
                    oldFilePath = os.path.join(oldFolderPath, file).replace('\\','/')  # 原图片文件夹路径
                    print('oldFilePath', oldFilePath)
                    outputfilename.write(oldFilePath)
                    outputfilename.write('\r\n')

                    shutil.copy(oldFilePath, newFolderPath)

CombineFile()
outputfilename.close()

### 删除空文件夹

In [ ]:
# delete_empty_folder
import os
for root, dirs, files in os.walk('../dataset', topdown=False):
    if not files and not dirs:
        os.rmdir(root)